In [33]:
from evaluate import *
from generate_data import *
import time

In [2]:
        
scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "conflict/enable": False,
                   "presolving/maxrounds": 0,
                   "presolving/maxrestarts": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   "separating/maxcuts": 0,
                   "propagating/maxroundsroot" : 0,
                   "lp/presolving" : False,
                  }

# generate_dataset(scip_parameters,path = "DataSetEvaluate/",nb_cons = [800,1000],nb_var = [800,1000],density = [0.1,0.2],nb_instance=20,only_problem=True)

In [3]:
def is_realisable(solu,A):
    size_var = len(solu)
    size_cons = A.shape[0] - size_var
    Aux = A[:size_var,size_cons:]
    for i,v in enumerate(solu):
        if v:
            for i_cons in np.where(Aux[i]!=0)[0][::-1]:
                Aux = np.delete(Aux, i_cons, axis=1)
    return Aux.shape[1]

In [13]:
path_set = os.listdir("DataSetEvaluate/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    lp = "DataSetEvaluate/"+filename
    train_name = "SC1000_BCE_3MLP_50TailleH_1batch_size_layernorm+cat"
    aux = []
    time_start=time.time()
    model_path = "model/"+train_name
    criterion = nn.BCELoss()
    net = VariablePredictor(24,50,3)
    net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
    res_her,res_gnn,A = optimize(scip_parameters,net,lp)
    time_gnn = time.time() - time_start
    
    time_start=time.time()
    solver = ecole.scip.Model.from_file(lp)
    aspyscip = solver.as_pyscipopt()
    aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
    aspyscip.optimize()
    res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
    time_scip = time.time() - time_start
    gap_scip = aspyscip.getGap()
    
    aux.append(is_realisable(res_gnn,A))
    if aux[0] != 0:
        aux.append(1)
    else:
        for i,v in enumerate(res_gnn):
            if v != res_scip[i]:
                aspyscip.setSolVal(aspyscip.getBestSol(),aspyscip.getVars()[i],v)
        aux.append(abs(aspyscip.getDualbound() - aspyscip.getObjVal())/min(aspyscip.getDualbound(),aspyscip.getObjVal()))
    
    for i,v in enumerate(res_her):
        aspyscip.setSolVal(aspyscip.getBestSol(),aspyscip.getVars()[i],v)
    aux.append(abs(aspyscip.getDualbound() - aspyscip.getObjVal())/min(aspyscip.getDualbound(),aspyscip.getObjVal()))
    aux.append(gap_scip)
    aux.append(time_gnn)    
    aux.append(time_scip)
    hist.append(aux)
    print("Instance %d|ConsResteGnn=%f,Gap_GNN=%f,Gap_Her=%f,gap_SCIP=%f \n temp_GNN=%f,temp_SCIP=%f"\
              %(idx,aux[0],aux[1],aux[2],aux[3],aux[4],aux[5]))
    idx+=1
np.mean(hist, axis=0)
hist = np.rot90(hist, -1)


pressed CTRL-C 1 times (5 times for forcing termination)
Instance 0|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,gap_SCIP=0.000000 
 temp_GNN=2.478720,temp_SCIP=9.941925
Instance 1|ConsResteGnn=195.000000,Gap_GNN=1.000000,Gap_Her=262.307692,gap_SCIP=0.000000 
 temp_GNN=3.921293,temp_SCIP=7.748015
Instance 2|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.067308,gap_SCIP=0.000000 
 temp_GNN=2.549265,temp_SCIP=16.655656
Instance 3|ConsResteGnn=0.000000,Gap_GNN=0.021053,Gap_Her=0.021053,gap_SCIP=0.000000 
 temp_GNN=3.272102,temp_SCIP=30.993094
Instance 4|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,gap_SCIP=0.000000 
 temp_GNN=2.110345,temp_SCIP=7.841611
Instance 5|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,gap_SCIP=0.000000 
 temp_GNN=3.121619,temp_SCIP=5.926083
Instance 6|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,gap_SCIP=0.000000 
 temp_GNN=1.473480,temp_SCIP=7.697986
Instance 7|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,ga

Instance 65|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,gap_SCIP=0.000000 
 temp_GNN=1.768055,temp_SCIP=7.510242
Instance 66|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,gap_SCIP=0.000000 
 temp_GNN=1.900949,temp_SCIP=3.588712
Instance 67|ConsResteGnn=192.000000,Gap_GNN=1.000000,Gap_Her=124.406977,gap_SCIP=0.000000 
 temp_GNN=2.829031,temp_SCIP=2.247406
Instance 68|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,gap_SCIP=0.000000 
 temp_GNN=2.001905,temp_SCIP=7.750211
Instance 69|ConsResteGnn=195.000000,Gap_GNN=1.000000,Gap_Her=104.062500,gap_SCIP=0.000000 
 temp_GNN=3.346047,temp_SCIP=31.552151
Instance 70|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,gap_SCIP=0.000000 
 temp_GNN=1.365798,temp_SCIP=1.350000
Instance 71|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,gap_SCIP=0.000000 
 temp_GNN=2.007150,temp_SCIP=23.667434
Instance 72|ConsResteGnn=209.000000,Gap_GNN=1.000000,Gap_Her=102.311111,gap_SCIP=0.000000 
 temp_GNN=3.314460,temp_

Instance 130|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,gap_SCIP=0.000000 
 temp_GNN=3.314890,temp_SCIP=9.061887
Instance 131|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,gap_SCIP=0.000000 
 temp_GNN=2.458184,temp_SCIP=6.602637
Instance 132|ConsResteGnn=0.000000,Gap_GNN=0.026549,Gap_Her=0.026549,gap_SCIP=0.000000 
 temp_GNN=1.971621,temp_SCIP=18.836202
Instance 133|ConsResteGnn=2.000000,Gap_GNN=1.000000,Gap_Her=0.000000,gap_SCIP=0.000000 
 temp_GNN=3.883520,temp_SCIP=27.637114
Instance 134|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.053030,gap_SCIP=0.000000 
 temp_GNN=3.477475,temp_SCIP=66.918199
Instance 135|ConsResteGnn=0.000000,Gap_GNN=0.023256,Gap_Her=0.007752,gap_SCIP=0.000000 
 temp_GNN=2.589474,temp_SCIP=86.051053
Instance 136|ConsResteGnn=0.000000,Gap_GNN=0.007812,Gap_Her=0.007812,gap_SCIP=0.000000 
 temp_GNN=2.021039,temp_SCIP=14.330105
Instance 137|ConsResteGnn=0.000000,Gap_GNN=0.019417,Gap_Her=0.019417,gap_SCIP=0.000000 
 temp_GNN=1.485653,temp_S

array([48.8375    ,  0.26007585, 48.87910804,  0.        ,  2.78709078,
       18.97627461])

In [32]:
print("Pourcentage de solutions realisables de GNN :",len(np.where(hist[0] == 0)[0])/len(hist[0]))
print("Gap des solutions realisables de GNN :",(sum(hist[1])-len(np.where(hist[1] == 1.0)[0]))/len(np.where(hist[1] != 1.0)[0]))
print("Pourcentage de solutions bonnes de GNN+Heuristique",len(np.where(hist[2]  < 1)[0])/len(hist[2]))
print("Gap des solutions de GNN+Heuristique:",(sum([i if i < 1 else 0 for i in hist[2]])/len(np.where(hist[2]  < 1)[0])))
print("Temps pour GNN:%f"%np.mean(hist[4]))
print("Temps pour SICP:%f"%np.mean(hist[5]))



Pourcentage de solutions realisables de GNN : 0.74375
Gap des solutions realisables de GNN : 0.005143997766360004
Pourcentage de solutions bonnes de GNN+Heuristique 0.75
Gap des solutions de GNN+Heuristique: 0.007205141606349676
Temps pour GNN:2.787091
Temps pour SICP:18.976275


In [43]:
path_set = os.listdir("DataSet_CFL/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    lp = "DataSet_CFL/"+filename+"/problem.lp"
    train_name = "CFL30_BCE_7MLP_50TailleH_1batch_size_layernorm+cat"
    aux = []
    time_start=time.time()
    model_path = "model/"+train_name
    criterion = nn.BCELoss()
    net = VariablePredictor(24,50,7)
    net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
    res_her,res_gnn,A = optimize(scip_parameters,net,lp)
    time_gnn = time.time() - time_start
    
    time_start=time.time()
    solver = ecole.scip.Model.from_file(lp)
    aspyscip = solver.as_pyscipopt()
    aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
    aspyscip.optimize()
    res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
    time_scip = time.time() - time_start
    gap_scip = aspyscip.getGap()
    
    aux.append(is_realisable(res_gnn,A))
    if aux[0] != 0:
        aux.append(1)
    else:
        for i,v in enumerate(res_gnn):
            if v != res_scip[i]:
                aspyscip.setSolVal(aspyscip.getBestSol(),aspyscip.getVars()[i],v)
        aux.append(abs(aspyscip.getDualbound() - aspyscip.getObjVal())/min(aspyscip.getDualbound(),aspyscip.getObjVal()))
    
    for i,v in enumerate(res_her):
        aspyscip.setSolVal(aspyscip.getBestSol(),aspyscip.getVars()[i],v)
    aux.append(abs(aspyscip.getDualbound() - aspyscip.getObjVal())/min(aspyscip.getDualbound(),aspyscip.getObjVal()))
    aux.append(gap_scip)
    aux.append(time_gnn)    
    aux.append(time_scip)
    hist.append(aux)
    print("Instance %d|ConsResteGnn=%f,Gap_GNN=%f,Gap_Her=%f,gap_SCIP=%f \n temp_GNN=%f,temp_SCIP=%f"\
              %(idx,aux[0],aux[1],aux[2],aux[3],aux[4],aux[5]))
    idx+=1
np.mean(hist, axis=0)
hist = np.rot90(hist, -1)

Instance 0|ConsResteGnn=1256.000000,Gap_GNN=1.000000,Gap_Her=14.705796,gap_SCIP=0.000000 
 temp_GNN=11.042413,temp_SCIP=12.094124
Instance 1|ConsResteGnn=1306.000000,Gap_GNN=1.000000,Gap_Her=14.181706,gap_SCIP=0.000000 
 temp_GNN=10.989495,temp_SCIP=15.819539
Instance 2|ConsResteGnn=1073.000000,Gap_GNN=1.000000,Gap_Her=11.109676,gap_SCIP=0.000000 
 temp_GNN=8.311065,temp_SCIP=23.224016
Instance 3|ConsResteGnn=869.000000,Gap_GNN=1.000000,Gap_Her=11.786579,gap_SCIP=0.000000 
 temp_GNN=4.490613,temp_SCIP=5.050774
Instance 4|ConsResteGnn=812.000000,Gap_GNN=1.000000,Gap_Her=11.628297,gap_SCIP=0.000000 
 temp_GNN=4.218256,temp_SCIP=17.118878
Instance 5|ConsResteGnn=1260.000000,Gap_GNN=1.000000,Gap_Her=15.110521,gap_SCIP=0.000000 
 temp_GNN=11.108387,temp_SCIP=1.953540
Instance 6|ConsResteGnn=1285.000000,Gap_GNN=1.000000,Gap_Her=12.616077,gap_SCIP=0.000000 
 temp_GNN=9.530438,temp_SCIP=21.483623
Instance 7|ConsResteGnn=1260.000000,Gap_GNN=1.000000,Gap_Her=14.313412,gap_SCIP=0.000000 
 temp_GN


KeyboardInterrupt

